In [4]:
import sys
import os
import numpy as np
import tensorflow as tf
# 文件夹
##############
# 文件夹
bpseq_old_folder = './bpseqFiles/bpseqFiles'
sta_old_folder = './staFiles/staFiles'

bpseq_dbn_new_folder = '../data/dbn_bpseqFiles/'
bpseq_flag_new_folder = '../data/flag_bpseqFiles/'
bpseq_test_new_folder = '../data/bpseqFiles/'

# 文件
### dbn_output_forge_file 归并
# stem_tab_file 表文件
# dbn_output_forge_file = './data/rna.dbn.txt'
# flag_output_forge_file = './data/rna.flag.txt'

#############
# 写表文件
stem_tab_file = '../table/stem.txt'
dbn_tab_file = '../table/dbn.txt'
flag_tab_file = '../table/flag.txt'
#
#############

###############
# 把所有文件写到以下的目录中
train_folder = '../data/train/'
test_folder = '../data/test/'
vaild_folder = '../data/vaild/'

dbn_file_in ='dbn.in.txt'
dbn_file_out ='dbn.out.txt'

flag_file_in ='flag.in.txt'
flag_file_out = 'flag.out.txt'
#
#################

if not os.path.exists(bpseq_dbn_new_folder):
    os.makedirs(bpseq_dbn_new_folder)
if not os.path.exists(bpseq_flag_new_folder):
    os.makedirs(bpseq_flag_new_folder)
    
if not os.path.exists(train_folder):
    os.makedirs(train_folder)
if not os.path.exists(test_folder):
    os.makedirs(test_folder)
if not os.path.exists(vaild_folder):
    os.makedirs(vaild_folder)
    
if not os.path.exists("../table"):
    os.makedirs("../table")

/data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/home/chenzhiyuan/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/home/chenzhiyuan/anaco

In [5]:
# 定义的轮赌盘算法 作为数据分类的依据
from __future__ import division
import numpy as np
import random,pdb
import operator

weights=[20,20,60]

def roulette_selection(weights):
        '''performs weighted selection or roulette wheel selection on a list
        and returns the index selected from the list'''

        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random.random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value

In [6]:

####引入进度条配置
from progressbar import *               # just a simple progress bar
##progressbar2的配置
widgets = ['Running: ', Percentage(), ' ', Bar(marker='*',left='[',right=']'),
           ' ', ETA(), ' ', FileTransferSpeed()] #see docs for other options
pbar = ProgressBar(widgets=widgets)
tf.logging.set_verbosity(tf.logging.INFO)


In [7]:
####清理脏数据，并重新保存####
#  
# bpseq_old_files 是原来的文件
# bpseq_dbn_new_files 修改之后的带括号的文件
# bpseq_flag_new_files 修改之后 01 的文件
bpseq_old_files=[] 
bpseq_dbn_new_files=[]
bpseq_flag_new_files=[]

for root, dirs, files in os.walk(bpseq_old_folder, topdown=False):
    print(root)
    bpseq_old_files =    [os.path.join(root, name) for name in files]
    bpseq_dbn_new_files =  [os.path.join(bpseq_dbn_new_folder, name) for name in files]
    bpseq_flag_new_files =  [os.path.join(bpseq_flag_new_folder, name) for name in files]
    sta_files =              [os.path.join(sta_old_folder,name[:-6]+'.sta')for name in files]

print("读取文件夹...",bpseq_old_files[1:3])
print("写文件夹...",bpseq_dbn_new_files[1:3])
print("写文件夹...",bpseq_flag_new_files[1:3])

./bpseqFiles/bpseqFiles
读取文件夹... ['./bpseqFiles/bpseqFiles/bpRNA_CRW_32951.bpseq', './bpseqFiles/bpseqFiles/bpRNA_CRW_42904.bpseq']
写文件夹... ['../data/dbn_bpseqFiles/bpRNA_CRW_32951.bpseq', '../data/dbn_bpseqFiles/bpRNA_CRW_42904.bpseq']
写文件夹... ['../data/flag_bpseqFiles/bpRNA_CRW_32951.bpseq', '../data/flag_bpseqFiles/bpRNA_CRW_42904.bpseq']


In [8]:
pbar = ProgressBar(widgets=widgets)
for (old_file,sta_file,dbn_output_file,flag_output_file) in pbar(zip(bpseq_old_files,sta_files,bpseq_dbn_new_files,bpseq_flag_new_files)):# 保存文件名，碱基，序号，匹配号

    with open(sta_file,'r') as fr:# 找到第一个可用的那一行数据
        start_line_num = 0
        dbn_line_offset = 1
        sta_lines =fr.readlines()

        while(sta_lines[start_line_num][0]=='#'):
            start_line_num += 1
        dbn_line = sta_lines[start_line_num + dbn_line_offset]
    
    with open(old_file,'r') as fr:
        start_line_num = 0
        dbn_new_lines  = ''
        flag_new_lines  = ''
        old_lines    = fr.readlines()
        for idx,old_line in enumerate(old_lines):
            if old_line[0]=='#':# 如果是文件注释
                dbn_new_line = old_line
                flag_new_line = old_line
                dbn_new_lines+=dbn_new_line
                flag_new_lines+=flag_new_line
                start_line_num += 1
            else:
                order,base,match = old_line.strip('\n').split(' ')

                base = base.upper()
                match_i = int(match)
                dbn = ''
                if base not in {'A','C','G','U'} and match_i>0:
                    trg_order,trg_base,trg_match = old_lines[match_i+start_line_num-1].strip('\n').split(' ')
                    if trg_base == 'A':base = 'U'
                    elif trg_base == 'C':base = 'G'
                    elif trg_base == 'U':base = np.random.choice(['A','G'])
                    elif trg_base == 'G':base = np.random.choice(['C','U'])
                if int(match)>0:
                    match_flag ='1'
                    if order < match:dbn = '('
                    else:dbn = ')'
                else: 
                    match_flag = '0'
                    dbn = '.'
#                 dbn_new_line = order+' '+base+' '+match+' '+dbn_line[idx-start_line_num]+'\n'  多种dbn 如果要使用请注释下一行
                dbn_new_line = order+' '+base+' '+match+' '+dbn+'\n'
                flag_new_line = order+' '+base+' '+match+' '+match_flag+'\n'
                dbn_new_lines+=dbn_new_line
                flag_new_lines+=flag_new_line
    with open(dbn_output_file,'w') as fw1 :
        fw1.writelines(dbn_new_lines)
    with open(flag_output_file,'w') as fw2:
        fw2.writelines(flag_new_lines)



Running: N/A% [*                                     ] Time:  0:03:29 488.0 B/s


In [9]:
###
#将大量不同到碱基对，合并到一个文件中，并且将输入输出分开
# 设置文件到上限和显现
toplimit = 250
downlimit = 100


def save_to_one(bpseq_new_folder,output_file_in,output_file_out,toplimit,downlimit):
    #将文件合并保存到output_file
    for root, dirs, files in os.walk(bpseq_new_folder, topdown=False):
        bpseq_files = [os.path.join(root, name) for name in files]

    pbar = ProgressBar(widgets=widgets)
    for file in pbar(bpseq_files):# 保存文件名，碱基，序号，匹配号
        with open(file,'r') as fr:
            lines = fr.readlines()
            if len(lines)>toplimit or len(lines)<downlimit:
                continue
            file_name = fr.name.strip('\n')
            start = 0
            file_info = 'None'
            while lines[start][0] == '#':
                start += 1
            if start>0:
                file_info = lines[start-1][1:].strip('\n')
            bases = ''
            order_num = ''
            match_num = ''
            marks = ''
            for line in lines[start:-1]:
                try:
                    order,base,match,mark = line.strip('\n').split(' ')#顺序，碱基，匹配信息，dbn，匹配标志
                except:
                    print('[Error]: from %s in %s'%(file_name,line))
                bases += base+' '
                order_num += order+' '
                match_num += match+' '
                marks += mark+' '

        # 轮盘算法 区分比例
        weights=[15,15,70]
        folder_class =  roulette_selection(weights)
        if folder_class == 2:
            folder = train_folder
        elif folder_class == 1:
            folder = vaild_folder
        else:
            folder = test_folder
        
        with open(folder + output_file_in,'a+') as fin:
            _line=bases+'\n'
            fin.write(_line)
        
        with open(folder + output_file_out,'a+') as fout:
            _line = marks+'\n'
            fout.write(_line)
        
            
save_to_one(bpseq_dbn_new_folder,dbn_file_in,dbn_file_out,toplimit,downlimit)
save_to_one(bpseq_flag_new_folder,flag_file_in,flag_file_out,toplimit,downlimit)
# save_to_one(bpseq_dbn_new_folder,flag_output_file)


Running: 100% [************************************] Time:  0:00:10   9.5 KiB/s
Running: 100% [************************************] Time:  0:00:09  10.1 KiB/s


In [10]:
# 生成词表文件
from enum import Enum
Type = Enum("Type",("feature","label"))
def generate_stem_file(input_seg_file,output_stem_file,model):
    #生产词表
    with open(input_seg_file,'r') as f:
        lines = f.readlines()
        word_dict = { }
        for line in lines:
            bases = line 
            for word in bases.split():
                word_dict.setdefault(word,0)
                word_dict[word] += 1
        sorted_word_dict = sorted(word_dict.items(),key = lambda d:d[1], reverse = True)
    with open(output_stem_file,'w') as f :
        if model==Type.label:
            f.write('<UNK>\t1000001\n')
            f.write('<PAD>\t1000000\n')
        elif model == Type.feature:
            f.write('<PAD>\t1000000\n')
        for item in sorted_word_dict:
            f.write('%s\t%s\n'%(item[0],item[1]))
            
generate_stem_file(train_folder + dbn_file_in,stem_tab_file,Type.label)
generate_stem_file(train_folder + dbn_file_out,dbn_tab_file,Type.feature)
generate_stem_file(train_folder + flag_file_out,flag_tab_file,Type.feature)


In [11]:
Type.label

<Type.label: 2>

## 